# imports

In [1]:
import warnings
warnings.simplefilter('ignore')

import gc

import numpy as np
import pandas as pd

pd.set_option('max_columns', None)
pd.set_option('max_rows', None)
pd.set_option('max_colwidth', 100)

from tqdm import tqdm
tqdm.pandas()
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import accuracy_score

from gensim.models import Word2Vec

import xgboost as xgb

# load data

In [2]:
train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')

match = pd.read_csv('../data/matches.csv')
role = pd.read_csv('../data/role.csv')

train.shape, test.shape, match.shape, role.shape

((146022, 2), (36506, 1), (182528, 8), (1825280, 61))

In [3]:
df = pd.concat([train, test]).reset_index(drop=True)
df = pd.merge(df, match[['id', 'duration']], on='id')
df.head()

,id,label,duration
0,0,0.0,1873
1,1,0.0,1817
2,2,1.0,1430
3,3,1.0,1574
4,4,0.0,1709


# role.csv FE

In [4]:
role = role.fillna(0.0)
role.loc[role['wardsbought'] == '\\N', 'wardsbought'] = 99
role['wardsbought'] = role['wardsbought'].astype('float64')

role['kDa'] = (role['kills'] + role['assists']) / (role['deaths'] + 1e-3)
role['heal_dmg_ratio'] = role['totheal'] / (role['totdmgtaken'] + 1e-3)
role['gold_earn_spent_diff'] = role['goldearned'] - role['goldspent']
role['truedmgtochamp_ratio'] = role['truedmgtochamp'] / (role['totdmgtochamp'] + 1e-3)
role['dmgtoturrets_ratio']  = role['dmgtoturrets'] / (role['dmgtoobj'] + 1e-3)

In [5]:
role.head()

,matchid,player,championid,ss1,ss2,role,position,item1,item2,item3,item4,item5,item6,trinket,kills,deaths,assists,largestkillingspree,largestmultikill,killingsprees,longesttimespentliving,doublekills,triplekills,quadrakills,pentakills,legendarykills,totdmgdealt,magicdmgdealt,physicaldmgdealt,truedmgdealt,largestcrit,totdmgtochamp,magicdmgtochamp,physdmgtochamp,truedmgtochamp,totheal,totunitshealed,dmgselfmit,dmgtoobj,dmgtoturrets,visionscore,timecc,totdmgtaken,magicdmgtaken,physdmgtaken,truedmgtaken,goldearned,goldspent,turretkills,inhibkills,totminionskilled,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood,kDa,heal_dmg_ratio,gold_earn_spent_diff,truedmgtochamp_ratio,dmgtoturrets_ratio
0,0,1,222,4,7,DUO_CARRY,BOT,3140.0,3046.0,3085.0,3031.0,3033.0,3047.0,3363.0,16.0,4.0,6.0,12.0,5.0,2.0,492.0,3.0,1.0,1.0,1.0,0.0,196527.0,3671.0,189040.0,3814.0,678.0,35513.0,1411.0,32717.0,1384.0,4103.0,4.0,11592.0,11225.0,4625.0,19.0,0.0,14606.0,3667.0,10646.0,292.0,16657.0,14400.0,1.0,0.0,247.0,12.0,5.0,7.0,552.0,17.0,2.0,99.0,10.0,4.0,0.0,5.498625,0.280912,2257.0,0.038972,0.412027
1,0,2,43,4,3,DUO_SUPPORT,BOT,2301.0,3158.0,3222.0,3114.0,3504.0,2055.0,3364.0,1.0,4.0,23.0,0.0,1.0,0.0,795.0,0.0,0.0,0.0,0.0,0.0,22426.0,17869.0,3741.0,814.0,0.0,13941.0,11391.0,1735.0,814.0,596.0,1.0,10283.0,2280.0,1561.0,64.0,0.0,13926.0,3117.0,10594.0,214.0,10432.0,9580.0,0.0,0.0,13.0,0.0,0.0,0.0,222.0,15.0,6.0,99.0,24.0,5.0,0.0,5.998500,0.042798,852.0,0.058389,0.684649
2,0,3,113,4,11,NONE,JUNGLE,3065.0,1401.0,1028.0,3158.0,3110.0,2055.0,3340.0,5.0,9.0,12.0,2.0,1.0,1.0,486.0,0.0,0.0,0.0,0.0,0.0,122293.0,45678.0,70626.0,5987.0,0.0,12500.0,4585.0,5841.0,2073.0,7908.0,1.0,45473.0,5279.0,328.0,28.0,0.0,34487.0,9448.0,23858.0,1179.0,11398.0,11700.0,0.0,0.0,34.0,95.0,56.0,39.0,1113.0,15.0,5.0,99.0,13.0,3.0,0.0,1.888679,0.229304,-302.0,0.165840,0.062133
3,0,4,126,4,12,SOLO,TOP,1055.0,3142.0,3042.0,3047.0,3071.0,1038.0,3340.0,4.0,5.0,6.0,3.0,1.0,1.0,233.0,0.0,0.0,0.0,0.0,0.0,151960.0,19903.0,124656.0,7401.0,0.0,20426.0,3612.0,16132.0,681.0,1984.0,1.0,10750.0,20051.0,7881.0,13.0,0.0,15935.0,2387.0,12788.0,760.0,11783.0,11200.0,5.0,2.0,179.0,13.0,4.0,9.0,215.0,16.0,0.0,99.0,8.0,1.0,0.0,1.999600,0.124506,583.0,0.033340,0.393048
4,0,5,103,14,4,SOLO,MID,3020.0,3165.0,3146.0,3089.0,1052.0,0.0,3340.0,11.0,4.0,7.0,4.0,1.0,3.0,771.0,0.0,0.0,0.0,0.0,0.0,128293.0,72665.0,18425.0,37202.0,0.0,26773.0,17338.0,1804.0,7630.0,4071.0,1.0,6363.0,2738.0,1638.0,29.0,0.0,13653.0,5551.0,7881.0,220.0,13488.0,12335.0,0.0,0.0,206.0,8.0,6.0,2.0,67.0,16.0,2.0,99.0,12.0,2.0,0.0,4.498875,0.298176,1153.0,0.284989,0.598247


In [6]:
def emb(df, c1, c2):
    emb_size = 4

    tmp = df.groupby(c1, as_index=False)[c2].agg(
        {'{}_{}_list'.format(c1, c2): list})
    sentences = tmp['{}_{}_list'.format(c1, c2)].values.tolist()
    del tmp['{}_{}_list'.format(c1, c2)]

    words = []
    for i in range(len(sentences)):
        words += sentences[i]
        sentences[i] = [str(x) for x in sentences[i]]
    words = list(set(words))

    model = Word2Vec(sentences, size=emb_size, window=5,
                     min_count=5, sg=0, hs=1, seed=2019)

    emb_matrix = []
    for w in words:
        w = str(w)
        if w in model:
            emb_matrix.append(model[w])
        else:
            emb_matrix.append([0] * emb_size)

    df_emb = pd.DataFrame(emb_matrix)
    df_emb.columns = ['{}_emb_{}'.format(
        c2, i) for i in range(emb_size)]

    df_emb[c2] = words

    del model, emb_matrix, sentences
    return df_emb


for c1, c2 in [['matchid', 'role'], ['matchid', 'position'], 
               ['matchid', 'item1'], ['matchid', 'item2'], 
               ['matchid', 'item3'], ['matchid', 'item4'],
               ['matchid', 'item5'], ['matchid', 'item6']]:
    role = role.merge(emb(role, c1, c2), on=c2, how='left')

role.head()

,matchid,player,championid,ss1,ss2,role,position,item1,item2,item3,item4,item5,item6,trinket,kills,deaths,assists,largestkillingspree,largestmultikill,killingsprees,longesttimespentliving,doublekills,triplekills,quadrakills,pentakills,legendarykills,totdmgdealt,magicdmgdealt,physicaldmgdealt,truedmgdealt,largestcrit,totdmgtochamp,magicdmgtochamp,physdmgtochamp,truedmgtochamp,totheal,totunitshealed,dmgselfmit,dmgtoobj,dmgtoturrets,visionscore,timecc,totdmgtaken,magicdmgtaken,physdmgtaken,truedmgtaken,goldearned,goldspent,turretkills,inhibkills,totminionskilled,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood,kDa,heal_dmg_ratio,gold_earn_spent_diff,truedmgtochamp_ratio,dmgtoturrets_ratio,role_emb_0,role_emb_1,role_emb_2,role_emb_3,position_emb_0,position_emb_1,position_emb_2,position_emb_3,item1_emb_0,item1_emb_1,item1_emb_2,item1_emb_3,item2_emb_0,item2_emb_1,item2_emb_2,item2_emb_3,item3_emb_0,item3_emb_1,item3_emb_2,item3_emb_3,item4_emb_0,item4_emb_1,item4_emb_2,item4_emb_3,item5_emb_0,item5_emb_1,item5_emb_2,item5_emb_3,item6_emb_0,item6_emb_1,item6_emb_2,item6_emb_3
0,0,1,222,4,7,DUO_CARRY,BOT,3140.0,3046.0,3085.0,3031.0,3033.0,3047.0,3363.0,16.0,4.0,6.0,12.0,5.0,2.0,492.0,3.0,1.0,1.0,1.0,0.0,196527.0,3671.0,189040.0,3814.0,678.0,35513.0,1411.0,32717.0,1384.0,4103.0,4.0,11592.0,11225.0,4625.0,19.0,0.0,14606.0,3667.0,10646.0,292.0,16657.0,14400.0,1.0,0.0,247.0,12.0,5.0,7.0,552.0,17.0,2.0,99.0,10.0,4.0,0.0,5.498625,0.280912,2257.0,0.038972,0.412027,0.238912,0.445466,-1.507654,1.912629,0.090160,-0.030572,0.745416,1.249515,-0.138577,0.185716,-0.503985,2.585266,0.429550,0.433422,-1.220563,1.254829,0.012643,-0.072872,-2.062411,-0.563358,-2.027131,-0.757887,-1.048383,-0.445943,-2.937870,1.706504,3.558085,0.278106,0.456865,0.810946,1.556836,0.890126
1,0,2,43,4,3,DUO_SUPPORT,BOT,2301.0,3158.0,3222.0,3114.0,3504.0,2055.0,3364.0,1.0,4.0,23.0,0.0,1.0,0.0,795.0,0.0,0.0,0.0,0.0,0.0,22426.0,17869.0,3741.0,814.0,0.0,13941.0,11391.0,1735.0,814.0,596.0,1.0,10283.0,2280.0,1561.0,64.0,0.0,13926.0,3117.0,10594.0,214.0,10432.0,9580.0,0.0,0.0,13.0,0.0,0.0,0.0,222.0,15.0,6.0,99.0,24.0,5.0,0.0,5.998500,0.042798,852.0,0.058389,0.684649,1.073655,2.086555,0.826720,-0.375686,0.090160,-0.030572,0.745416,1.249515,-3.508702,2.469507,0.182503,-1.898940,-1.021182,-0.167990,-0.349632,-0.621413,-3.752770,-1.226515,3.260937,-0.033732,3.789571,0.330168,0.289867,1.559148,-1.522403,-2.380805,2.466575,-1.841774,0.961968,-0.802719,-0.791474,1.457654
2,0,3,113,4,11,NONE,JUNGLE,3065.0,1401.0,1028.0,3158.0,3110.0,2055.0,3340.0,5.0,9.0,12.0,2.0,1.0,1.0,486.0,0.0,0.0,0.0,0.0,0.0,122293.0,45678.0,70626.0,5987.0,0.0,12500.0,4585.0,5841.0,2073.0,7908.0,1.0,45473.0,5279.0,328.0,28.0,0.0,34487.0,9448.0,23858.0,1179.0,11398.0,11700.0,0.0,0.0,34.0,95.0,56.0,39.0,1113.0,15.0,5.0,99.0,13.0,3.0,0.0,1.888679,0.229304,-302.0,0.165840,0.062133,-0.860474,-1.117061,1.133705,0.070889,0.236856,1.268274,-1.519371,0.567367,-0.553419,0.908572,-0.761943,2.251202,-0.084364,0.482555,-1.106787,0.553996,2.183718,-0.391866,-0.628745,2.103669,0.320440,-0.926078,-0.016739,0.368482,-1.297882,-2.435590,3.704381,-0.176391,0.961968,-0.802719,-0.791474,1.457654
3,0,4,126,4,12,SOLO,TOP,1055.0,3142.0,3042.0,3047.0,3071.0,1038.0,3340.0,4.0,5.0,6.0,3.0,1.0,1.0,233.0,0.0,0.0,0.0,0.0,0.0,151960.0,19903.0,124656.0,7401.0,0.0,20426.0,3612.0,16132.0,681.0,1984.0,1.0,10750.0,20051.0,7881.0,13.0,0.0,15935.0,2387.0,12788.0,760.0,11783.0,11200.0,5.0,2.0,179.0,13.0,4.0,9.0,215.0,16.0,0.0,99.0,8.0,1.0,0.0,1.999600,0.124506,583.0,0.033340,0.393048,-0.089273,-0.569299,0.544138,0.277241,-1.898522,0.146334,0.569604,-1.245882,0.688708,-3.164435,0.782259,-0.586422,0.258705,0.238456,-0.762178,1.308857,-1.877421,0.162825,-1.667399,-1.449747,0.194492,-0.772000,-0.806485,-0.204496,-1.538343,0.688671,1.348589,0.083493,-0.659642,2.040898,0.196562,0.468211
4,0,5,103,14,4,SOLO,MID,3020.0,3165.0,3146.0,3089.0,1052.0,0.0,3340.0,11.0,4.0,7.0,4.0,1

## split to two teams 

In [7]:
role_team1 = role[role.player.isin([1,2,3,4,5])]
role_team2 = role[role.player.isin([6,7,8,9,10])]

In [8]:
role_team1.head()

,matchid,player,championid,ss1,ss2,role,position,item1,item2,item3,item4,item5,item6,trinket,kills,deaths,assists,largestkillingspree,largestmultikill,killingsprees,longesttimespentliving,doublekills,triplekills,quadrakills,pentakills,legendarykills,totdmgdealt,magicdmgdealt,physicaldmgdealt,truedmgdealt,largestcrit,totdmgtochamp,magicdmgtochamp,physdmgtochamp,truedmgtochamp,totheal,totunitshealed,dmgselfmit,dmgtoobj,dmgtoturrets,visionscore,timecc,totdmgtaken,magicdmgtaken,physdmgtaken,truedmgtaken,goldearned,goldspent,turretkills,inhibkills,totminionskilled,neutralminionskilled,ownjunglekills,enemyjunglekills,totcctimedealt,champlvl,pinksbought,wardsbought,wardsplaced,wardskilled,firstblood,kDa,heal_dmg_ratio,gold_earn_spent_diff,truedmgtochamp_ratio,dmgtoturrets_ratio,role_emb_0,role_emb_1,role_emb_2,role_emb_3,position_emb_0,position_emb_1,position_emb_2,position_emb_3,item1_emb_0,item1_emb_1,item1_emb_2,item1_emb_3,item2_emb_0,item2_emb_1,item2_emb_2,item2_emb_3,item3_emb_0,item3_emb_1,item3_emb_2,item3_emb_3,item4_emb_0,item4_emb_1,item4_emb_2,item4_emb_3,item5_emb_0,item5_emb_1,item5_emb_2,item5_emb_3,item6_emb_0,item6_emb_1,item6_emb_2,item6_emb_3
0,0,1,222,4,7,DUO_CARRY,BOT,3140.0,3046.0,3085.0,3031.0,3033.0,3047.0,3363.0,16.0,4.0,6.0,12.0,5.0,2.0,492.0,3.0,1.0,1.0,1.0,0.0,196527.0,3671.0,189040.0,3814.0,678.0,35513.0,1411.0,32717.0,1384.0,4103.0,4.0,11592.0,11225.0,4625.0,19.0,0.0,14606.0,3667.0,10646.0,292.0,16657.0,14400.0,1.0,0.0,247.0,12.0,5.0,7.0,552.0,17.0,2.0,99.0,10.0,4.0,0.0,5.498625,0.280912,2257.0,0.038972,0.412027,0.238912,0.445466,-1.507654,1.912629,0.090160,-0.030572,0.745416,1.249515,-0.138577,0.185716,-0.503985,2.585266,0.429550,0.433422,-1.220563,1.254829,0.012643,-0.072872,-2.062411,-0.563358,-2.027131,-0.757887,-1.048383,-0.445943,-2.937870,1.706504,3.558085,0.278106,0.456865,0.810946,1.556836,0.890126
1,0,2,43,4,3,DUO_SUPPORT,BOT,2301.0,3158.0,3222.0,3114.0,3504.0,2055.0,3364.0,1.0,4.0,23.0,0.0,1.0,0.0,795.0,0.0,0.0,0.0,0.0,0.0,22426.0,17869.0,3741.0,814.0,0.0,13941.0,11391.0,1735.0,814.0,596.0,1.0,10283.0,2280.0,1561.0,64.0,0.0,13926.0,3117.0,10594.0,214.0,10432.0,9580.0,0.0,0.0,13.0,0.0,0.0,0.0,222.0,15.0,6.0,99.0,24.0,5.0,0.0,5.998500,0.042798,852.0,0.058389,0.684649,1.073655,2.086555,0.826720,-0.375686,0.090160,-0.030572,0.745416,1.249515,-3.508702,2.469507,0.182503,-1.898940,-1.021182,-0.167990,-0.349632,-0.621413,-3.752770,-1.226515,3.260937,-0.033732,3.789571,0.330168,0.289867,1.559148,-1.522403,-2.380805,2.466575,-1.841774,0.961968,-0.802719,-0.791474,1.457654
2,0,3,113,4,11,NONE,JUNGLE,3065.0,1401.0,1028.0,3158.0,3110.0,2055.0,3340.0,5.0,9.0,12.0,2.0,1.0,1.0,486.0,0.0,0.0,0.0,0.0,0.0,122293.0,45678.0,70626.0,5987.0,0.0,12500.0,4585.0,5841.0,2073.0,7908.0,1.0,45473.0,5279.0,328.0,28.0,0.0,34487.0,9448.0,23858.0,1179.0,11398.0,11700.0,0.0,0.0,34.0,95.0,56.0,39.0,1113.0,15.0,5.0,99.0,13.0,3.0,0.0,1.888679,0.229304,-302.0,0.165840,0.062133,-0.860474,-1.117061,1.133705,0.070889,0.236856,1.268274,-1.519371,0.567367,-0.553419,0.908572,-0.761943,2.251202,-0.084364,0.482555,-1.106787,0.553996,2.183718,-0.391866,-0.628745,2.103669,0.320440,-0.926078,-0.016739,0.368482,-1.297882,-2.435590,3.704381,-0.176391,0.961968,-0.802719,-0.791474,1.457654
3,0,4,126,4,12,SOLO,TOP,1055.0,3142.0,3042.0,3047.0,3071.0,1038.0,3340.0,4.0,5.0,6.0,3.0,1.0,1.0,233.0,0.0,0.0,0.0,0.0,0.0,151960.0,19903.0,124656.0,7401.0,0.0,20426.0,3612.0,16132.0,681.0,1984.0,1.0,10750.0,20051.0,7881.0,13.0,0.0,15935.0,2387.0,12788.0,760.0,11783.0,11200.0,5.0,2.0,179.0,13.0,4.0,9.0,215.0,16.0,0.0,99.0,8.0,1.0,0.0,1.999600,0.124506,583.0,0.033340,0.393048,-0.089273,-0.569299,0.544138,0.277241,-1.898522,0.146334,0.569604,-1.245882,0.688708,-3.164435,0.782259,-0.586422,0.258705,0.238456,-0.762178,1.308857,-1.877421,0.162825,-1.667399,-1.449747,0.194492,-0.772000,-0.806485,-0.204496,-1.538343,0.688671,1.348589,0.083493,-0.659642,2.040898,0.196562,0.468211
4,0,5,103,14,4,SOLO,MID,3020.0,3165.0,3146.0,3089.0,1052.0,0.0,3340.0,11.0,4.0,7.0,4.0,1

## team stats 

In [9]:
new_cols = list()

for col in tqdm([
    'kills', 'deaths', 'assists', 
    'largestkillingspree', 'largestmultikill', 'killingsprees', 'longesttimespentliving',
    'doublekills', 'triplekills','quadrakills', 'pentakills', 'legendarykills', 
    'totdmgdealt', 'magicdmgdealt', 'physicaldmgdealt', 'truedmgdealt',
    'totdmgtochamp', 'magicdmgtochamp', 'physdmgtochamp', 'truedmgtochamp',
    'totheal', 'totunitshealed', 'dmgselfmit', 'dmgtoobj', 'dmgtoturrets',
    'visionscore', 'timecc', 'totdmgtaken', 'magicdmgtaken', 'physdmgtaken',
    'truedmgtaken', 'goldearned', 'goldspent', 'turretkills', 'inhibkills',
    'totminionskilled', 'neutralminionskilled', 'ownjunglekills',
    'enemyjunglekills', 'totcctimedealt', 'pinksbought',
    'wardsplaced', 'wardskilled', 'firstblood',
    'kDa', 'heal_dmg_ratio', 'gold_earn_spent_diff', 'truedmgtochamp_ratio', 'dmgtoturrets_ratio']):
    for method in ['sum', 'min', 'max', 'std', 'mean']:
        colname = f'{col}_{method}'
        role_team1[colname] = role_team1.groupby('matchid')[col].transform(method)
        role_team2[colname] = role_team2.groupby('matchid')[col].transform(method)
        new_cols.append(colname)

100%|██████████| 49/49 [04:07<00:00,  5.05s/it]


In [10]:
role_team1 = role_team1.drop_duplicates(subset=['matchid']).reset_index(drop=True)
role_team2 = role_team2.drop_duplicates(subset=['matchid']).reset_index(drop=True)

role_team1.shape, role_team2.shape

((182528, 343), (182528, 343))

## teams differ 

In [11]:
df_role = role_team1[['matchid']]

for col in tqdm(new_cols):
    df_role[f'team1_{col}'] = role_team1[col]
    df_role[f'team2_{col}'] = role_team2[col]
    df_role[f'teamdiff_{col}'] = df_role[f'team1_{col}'] - df_role[f'team2_{col}']
    df_role[f'teamdiff2_{col}'] = df_role[f'team2_{col}'] - df_role[f'team1_{col}']
    df_role[f'teamratio_{col}'] = df_role[f'team1_{col}'] / (df_role[f'team2_{col}'] + 1e-3)
    df_role[f'teamratio2_{col}'] = df_role[f'team2_{col}'] / (df_role[f'team1_{col}'] + 1e-3)

100%|██████████| 245/245 [00:07<00:00, 30.83it/s]


In [12]:
df_role = df_role.rename(columns={'matchid': 'id'})
df_role.head()

id  team1_kills_sum  team2_kills_sum  teamdiff_kills_sum  \
0   0             37.0             26.0                11.0   
1   1             29.0             26.0                 3.0   
2   2             14.0             37.0               -23.0   
3   3             21.0             28.0                -7.0   
4   4             26.0             16.0                10.0   

   teamdiff2_kills_sum  teamratio_kills_sum  teamratio2_kills_sum  \
0                -11.0             1.423022              0.702684   
1                 -3.0             1.115342              0.896521   
2                 23.0             0.378368              2.642668   
3                  7.0             0.749973              1.333270   
4                -10.0             1.624898              0.615361   

   team1_kills_min  team2_kills_min  teamdiff_kills_min  teamdiff2_kills_min  \
0              1.0              2.0                -1.0                  1.0   
1              3.0              0.0                 3.0                 -3.0   
2              1.0              3.0                -2.0                  2.0   
3              0.0              2.0                -2.0                  2.0   
4              2.0              2.0                 0.0                  0.0   

   teamratio_kills_min  teamratio2_kills_min  team1_kills_max  \
0             0.499750              1.998002             16.0   
1          3000.000000              0.000000             10.0   
2             0.333222              2.997003              6.0   
3             0.000000           2000.000000             10.0   
4             0.999500              0.999500              9.0   

   team2_kills_max  teamdiff_kills_max  teamdiff2_kills_max  \
0              7.0                 9.0                 -9.0   
1             11.0                -1.0                  1.0   
2             11.0                -5.0                  5.0   
3              8.0                 2.0                 -2.0   
4              6.0                 3.0                 -3.0   

   teamratio_kills_max  teamratio2_kills_max  team1_kills_std  \
0             2.285388              0.437473         6.024948   
1             0.909008              1.099890         2.588436   
2             0.545405              1.833028         2.049390   
3             1.249844              0.799920         4.207137   
4             1.499750              0.666593         3.114482   

   team2_kills_std  teamdiff_kills_std  teamdiff2_kills_std  \
0         2.489980            3.534968            -3.534968   
1         4.438468           -1.850032             1.850032   
2         3.361547           -1.312157             1.312157   
3         2.880972            1.326165            -1.326165   
4         1.643168            1.471315            -1.471315   

   teamratio_kills_std  teamratio2_kills_std  team1_kills_mean  \
0             2.418706              0.413210               7.4   
1             0.583051              1.714068               5.8   
2             0.609475              1.639467               2.8   
3             1.459812              0.684619               4.2   
4             1.894261              0.527420               5.2   

   team2_kills_mean  teamdiff_kills_mean  teamdiff2_kills_mean  \
0               5.2                  2.2                  -2.2   
1               5.2                  0.6                  -0.6   
2               7.4                 -4.6                   4.6   
3               5.6                 -1.4                   1.4   
4               3.2                  2.0                  -2.0   

   teamratio_kills_mean  teamratio2_kills_mean  team1_deaths_sum  \
0              1.422803               0.702608              26.0   
1              1.115170               0.896397              27.0   
2              0.378327               2.641914              37.0   
3              0.749866               1.333016              28.0   
4              1.624492               0.615266        

# Merge as one dataframe

In [13]:
df = pd.merge(df, df_role, on='id')

# FE on duration time

In [14]:
for col in ['largestkillingspree', 'largestmultikill', 'killingsprees', 'longesttimespentliving']:
    df[f'team1_{col}_mean_per_minute'] = df[f'team1_{col}_mean'] / (df['duration'] / 60)
    df[f'team2_{col}_mean_per_minute'] = df[f'team2_{col}_mean'] / (df['duration'] / 60)
    df[f'teamdiff_{col}_mean_per_minute'] = df[f'team1_{col}_mean_per_minute'] - df[f'team2_{col}_mean_per_minute']
    df[f'teamdiff2_{col}_mean_per_minute'] = df[f'team2_{col}_mean_per_minute'] - df[f'team1_{col}_mean_per_minute']
    df[f'teamratio_{col}_mean_per_minute'] = df[f'team1_{col}_mean_per_minute'] / (df[f'team2_{col}_mean_per_minute'] +1e-3)
    df[f'teamratio2_{col}_mean_per_minute'] = df[f'team2_{col}_mean_per_minute'] / (df[f'team1_{col}_mean_per_minute'] +1e-3)

# FE on OneVsOne

In [15]:
player1  =  role[role['player']==1].reset_index(drop=True)
player2  =  role[role['player']==2].reset_index(drop=True)
player3  =  role[role['player']==3].reset_index(drop=True)
player4  =  role[role['player']==4].reset_index(drop=True)
player5  =  role[role['player']==5].reset_index(drop=True)
player6  =  role[role['player']==6].reset_index(drop=True)
player7  =  role[role['player']==7].reset_index(drop=True)
player8  =  role[role['player']==8].reset_index(drop=True)
player9  =  role[role['player']==9].reset_index(drop=True)
player10 = role[role['player']==10].reset_index(drop=True)

df1 = player1[['matchid']]
for col in [col for col in player1.columns if col not in ['matchid', 'player', 'championid', 'role', 'position'] + [f'item{i}' for i in range(1,7)]]:
    df1[col] = player1[col] - player6[col]
    
df2 = player2[['matchid']]
for col in [col for col in player1.columns if col not in ['matchid', 'player', 'championid', 'role', 'position'] + [f'item{i}' for i in range(1,7)]]:
    df2[col] = player2[col] - player7[col]
    
df3 = player3[['matchid']]
for col in [col for col in player1.columns if col not in ['matchid', 'player', 'championid', 'role', 'position'] + [f'item{i}' for i in range(1,7)]]:
    df3[col] = player3[col] - player8[col]
    
df4 = player4[['matchid']]
for col in [col for col in player4.columns if col not in ['matchid', 'player', 'championid', 'role', 'position'] + [f'item{i}' for i in range(1,7)]]:
    df4[col] = player4[col] - player9[col]
    
df5 = player5[['matchid']]
for col in [col for col in player5.columns if col not in ['matchid', 'player', 'championid', 'role', 'position'] + [f'item{i}' for i in range(1,7)]]:
    df5[col] = player5[col] - player10[col]
    
df1.columns = ['matchid'] + [f'{col}_1' for col in df1.columns if col not in ['matchid']]
df2.columns = ['matchid'] + [f'{col}_2' for col in df2.columns if col not in ['matchid']]
df3.columns = ['matchid'] + [f'{col}_3' for col in df3.columns if col not in ['matchid']]
df4.columns = ['matchid'] + [f'{col}_4' for col in df4.columns if col not in ['matchid']]
df5.columns = ['matchid'] + [f'{col}_5' for col in df5.columns if col not in ['matchid']]

df = pd.concat([df, df1, df2, df3, df4, df5], axis=1)
to_drops = ['matchid']
df = df.drop(to_drops, axis=1)

for col in ['kills', 'deaths', 'assists', 'totdmgdealt', 'truedmgtochamp', 'totheal', 
            'dmgtoobj', 'dmgtoturrets', 'visionscore', 'totdmgtaken', 'goldearned', 'goldspent',
            'truedmgtaken', 'turretkills', 'champlvl', 'totdmgtochamp']:
    df[f'{col}_mean'] = df[[f'{col}_{i}' for i in range(1,6)]].mean(axis=1)
    df[f'{col}_std'] = df[[f'{col}_{i}' for i in range(1,6)]].std(axis=1)
    df[f'{col}_max'] = df[[f'{col}_{i}' for i in range(1,6)]].max(axis=1)
    df[f'{col}_min'] = df[[f'{col}_{i}' for i in range(1,6)]].min(axis=1)

In [16]:
df.head()

id  label  duration  team1_kills_sum  team2_kills_sum  teamdiff_kills_sum  \
0   0    0.0      1873             37.0             26.0                11.0   
1   1    0.0      1817             29.0             26.0                 3.0   
2   2    1.0      1430             14.0             37.0               -23.0   
3   3    1.0      1574             21.0             28.0                -7.0   
4   4    0.0      1709             26.0             16.0                10.0   

   teamdiff2_kills_sum  teamratio_kills_sum  teamratio2_kills_sum  \
0                -11.0             1.423022              0.702684   
1                 -3.0             1.115342              0.896521   
2                 23.0             0.378368              2.642668   
3                  7.0             0.749973              1.333270   
4                -10.0             1.624898              0.615361   

   team1_kills_min  team2_kills_min  teamdiff_kills_min  teamdiff2_kills_min  \
0              1.0              2.0                -1.0                  1.0   
1              3.0              0.0                 3.0                 -3.0   
2              1.0              3.0                -2.0                  2.0   
3              0.0              2.0                -2.0                  2.0   
4              2.0              2.0                 0.0                  0.0   

   teamratio_kills_min  teamratio2_kills_min  team1_kills_max  \
0             0.499750              1.998002             16.0   
1          3000.000000              0.000000             10.0   
2             0.333222              2.997003              6.0   
3             0.000000           2000.000000             10.0   
4             0.999500              0.999500              9.0   

   team2_kills_max  teamdiff_kills_max  teamdiff2_kills_max  \
0              7.0                 9.0                 -9.0   
1             11.0                -1.0                  1.0   
2             11.0                -5.0                  5.0   
3              8.0                 2.0                 -2.0   
4              6.0                 3.0                 -3.0   

   teamratio_kills_max  teamratio2_kills_max  team1_kills_std  \
0             2.285388              0.437473         6.024948   
1             0.909008              1.099890         2.588436   
2             0.545405              1.833028         2.049390   
3             1.249844              0.799920         4.207137   
4             1.499750              0.666593         3.114482   

   team2_kills_std  teamdiff_kills_std  teamdiff2_kills_std  \
0         2.489980            3.534968            -3.534968   
1         4.438468           -1.850032             1.850032   
2         3.361547           -1.312157             1.312157   
3         2.880972            1.326165            -1.326165   
4         1.643168            1.471315            -1.471315   

   teamratio_kills_std  teamratio2_kills_std  team1_kills_mean  \
0             2.418706              0.413210               7.4   
1             0.583051              1.714068               5.8   
2             0.609475              1.639467               2.8   
3             1.459812              0.684619               4.2   
4             1.894261              0.527420               5.2   

   team2_kills_mean  teamdiff_kills_mean  teamdiff2_kills_mean  \
0               5.2                  2.2                  -2.2   
1               5.2                  0.6                  -0.6   
2               7.4                 -4.6                   4.6   
3               5.6                 -1.4                   1.4   
4               3.2                  2.0                  -2.0   

   teamratio_kills_mean  teamratio2_kills_mean  team1_deaths_sum  \
0              1.422803               0.702608              26.0   
1              1.115170               0.896397              27.0   
2              0.378327               2.641914              37.0   
3              0.749

## Model and training 

In [17]:
train = df[df.label.notna()]
test = df[df.label.isna()]

train.shape, test.shape

((146022, 1996), (36506, 1996))

In [18]:
ycol = 'label'
feature_names = list(
    filter(lambda x: x not in [ycol, 'id'], train.columns))

model = xgb.XGBClassifier(num_leaves=64,
                          max_depth=8,
                          learning_rate=0.1,
                          n_estimators=500,
                          subsample=0.7,
                          feature_fraction=0.5,
                          reg_alpha=8,
                          reg_lambda=12,
                          random_state=1983,
                          tree_method='gpu_hist')

oof = []
prediction = test[['id']]
prediction[f'{ycol}_0'] = 0
prediction[f'{ycol}_1'] = 0
df_importance_list = []

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=1983)
for fold_id, (trn_idx, val_idx) in enumerate(kfold.split(train[feature_names], train[ycol])):
    X_train = train.iloc[trn_idx][feature_names]
    Y_train = train.iloc[trn_idx][ycol]

    X_val = train.iloc[val_idx][feature_names]
    Y_val = train.iloc[val_idx][ycol]

    print('\nFold_{} Training ================================\n'.format(fold_id+1))

    xgb_model = model.fit(X_train,
                          Y_train,
                          eval_set=[(X_train, Y_train), (X_val, Y_val)],
                          verbose=5000,
                          eval_metric='auc',
                          early_stopping_rounds=50)

    pred_val = xgb_model.predict_proba(X_val)
    df_oof = train.iloc[val_idx][['id', ycol]].copy()
    df_oof['label_0'] = pred_val[:,0]
    df_oof['label_1'] = pred_val[:,1]
    oof.append(df_oof)

    pred_test = xgb_model.predict_proba(test[feature_names])
    prediction[f'{ycol}_0'] += pred_test[:,0] / kfold.n_splits
    prediction[f'{ycol}_1'] += pred_test[:,1] / kfold.n_splits

    df_importance = pd.DataFrame({
        'column': feature_names,
        'importance': xgb_model.feature_importances_,
    })
    df_importance_list.append(df_importance)

    del xgb_model, pred_val, pred_test, X_train, Y_train, X_val, Y_val
    gc.collect()


Fold_1 Training ================================

[0]	validation_0-auc:0.996525	validation_1-auc:0.995862
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
Stopping. Best iteration:
[138]	validation_0-auc:0.999983	validation_1-auc:0.9998


Fold_2 Training ================================

[0]	validation_0-auc:0.996064	validation_1-auc:0.995973
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
Stopping. Best iteration:
[108]	validation_0-auc:0.99997	validation_1-auc:0.999852


Fold_3 Training ================================

[0]	validation_0-auc:0.996286	validation_1-auc:0.996055
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 50 rounds.
Stopping. Best iteration:
[239]	validatio

In [19]:
df_oof = pd.concat(oof)

score = accuracy_score(df_oof[ycol].astype('int'), np.argmax(df_oof[['label_0', 'label_1']].values, axis=1).astype('int'))
print('acc:', score)

acc: 0.993987207407103


# importance

In [20]:
df_importance = pd.concat(df_importance_list)
df_importance = df_importance.groupby(['column'])['importance'].agg(
    'mean').sort_values(ascending=False).reset_index()
df_importance

,column,importance
0,teamratio_goldearned_sum,0.207370
1,teamratio2_goldearned_mean,0.183104
2,teamratio_goldearned_mean,0.132808
3,teamratio2_dmgtoturrets_sum,0.019397
4,teamratio2_inhibkills_sum,0.006421
5,teamratio_turretkills_sum,0.004419
6,champlvl_mean,0.003389
7,teamratio2_turretkills_sum,0.002925
8,teamdiff_gold_earn_spent_diff_sum,0.002882
9,teamratio_inhibkills_sum,0.002570


# submit

In [22]:
sub = prediction.copy(deep=True)
sub['label'] = np.argmax(sub[[f'{ycol}_0', f'{ycol}_1']].values, axis=1).astype('int')

print(sub.label.value_counts())
sub[['id', 'label']].to_csv('submission_xgb.csv', index=False, header=False, encoding='utf-8') 

0    18591
1    17915
Name: label, dtype: int64


# save for blending 

In [24]:
df_oof.to_pickle('../data/oof_xgb.pkl')
prediction.to_pickle('../data/pred_xgb.pkl')